# Evaluation of Seq2Seq Models

In [1]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import datasets 

import pandas as pd

### Load Test Data and Preprocess

In [14]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

def tokenize_function(set):
    inputs = tokenizer(set["code"], max_length=512, padding="max_length", truncation=True, return_tensors="pt")
    with tokenizer.as_target_tokenizer():
       labels = tokenizer(set["docstring"], max_length=512, padding="max_length", truncation=True, return_tensors="pt")

    inputs["labels"] = labels["input_ids"]

    return inputs

In [15]:
test_data = datasets.load_dataset('json', data_files="D:\\PROJECT\\data\\CodeSearchNet\\pyjava\\test.jsonl")["train"]
tokenized_test = test_data.map(tokenize_function, remove_columns=test_data.column_names)
test_set = tokenized_test.shuffle()

Using custom data configuration default-0ac7eaced0986bb3
Reusing dataset json (C:\Users\Parry\.cache\huggingface\datasets\json\default-0ac7eaced0986bb3\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)
100%|██████████| 25873/25873 [00:52<00:00, 496.82ex/s]


## No Augmentation of Dataset

In [16]:
def runTest(model):
    bleu = datasets.load_metric('bleu')
    rouge = datasets.load_metric('rouge')
    meteor = datasets.load_metric('meteor')

    for input, reference in test_set:
        model_pred = model(input)
        bleu.add(predictions=model_pred, references=reference)
        rouge.add(predictions=model_pred, references=reference)
        meteor.add(predictions=model_pred, references=reference)
    
    return  {'bleu' : bleu.compute(), 'rouge' : rouge.compute(), 'meteor' : meteor.compute()}


In [17]:
tiny_path = "D:\\PROJECT\Models\\tiny\\model_trained\\checkpoint-310000"
small_path = ""
medium_path = ""

In [18]:
tiny_model = AutoModelForSeq2SeqLM.from_pretrained(tiny_path, 
    pad_token_id=1, 
    bos_token_id = 0, 
    eos_token_id = 2, 
    decoder_start_token_id = 0)

In [ ]:
small_model = AutoModelForSeq2SeqLM.from_pretrained(small_path, 
    pad_token_id=1, 
    bos_token_id = 0, 
    eos_token_id = 2, 
    decoder_start_token_id = 0)

In [ ]:
medium_model = AutoModelForSeq2SeqLM.from_pretrained(medium_path, 
    pad_token_id=1, 
    bos_token_id = 0, 
    eos_token_id = 2, 
    decoder_start_token_id = 0)

In [19]:
tiny_res = runTest(tiny_model)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Parry\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Parry\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Parry\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


AttributeError: 'list' object has no attribute 'size'